In [28]:
%run ../utils.ipynb

In [3]:
import os
import sys
from logging import DEBUG
from pathlib import Path

from dotenv import load_dotenv

from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-10 14:36:52,645 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-10 14:36:52,649 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-10 14:36:52,651 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-10 14:36:52,651 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-10 14:36:52,699 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-10 14:36:52,702 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-10 14:36:52,702 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


### Setup

In [4]:
auth_dir = f"{project_dir}/auth"

In [5]:
data_dir = f"{project_dir}/output"

In [26]:
data = Data(data_dir)

### Regenerate functions (infrequent use)

In [30]:
def write_league_standings(game_keys = Game_Keys, league_ids = League_Ids):
    """
    Generates JSON files containing YFPY Team objects that can represent the current or past standings, among other information

    By default will use the Game_Keys and League_Ids global variables from utils.ipynb, and attempt to generate standings for all seasons within those.

    Can be filtered by passing an specific seasons in game_keys
    """
    #TO DO: write this to be less specific in its arguments so it can be incorporated into a weekly pipeline
    
    for year, id in zip(game_keys['season'], game_keys['code']):
        print(f"Attempting to write year {year}")

        #make query representing a year of fantasy league
        yahoo_query = YahooFantasySportsQuery(
                auth_dir,
                league_id=league_ids[str(year)],
                game_code='nfl',
                game_id=id
            )
        try:
            data.save(file_name=f"standings_year_{year}", yf_query=yahoo_query.get_league_standings)
        except:
            print(f"Could not save data for year: {year}, code: {id}")
        else:
            pass

### Output data

In [27]:
# Uncomment to regenerate standings per modern era year if needed
# write_league_standings(game_keys[game_keys['season'] == 2023], league_ids)

Attempting to write year 2023


### Generate fresh standings dataframe

In [47]:
# Prepare a list of filenames only (no extensions) for use with the YFPY Data.load method
standings_filenames = [filename.replace('.json','') for filename in os.listdir(f"{project_dir}\\output\\standings_by_season\\") if '.json' in filename]

In [48]:
all_team_stats = []

# load each file (season) by name, results in a "Teams" object
for file in standings_filenames:
    working_file = data.load(f'standings_by_season/{file}')

    # loop through each Team object in Teams and serialize it so it can be appended as a dict 
    for i in working_file['teams']:
        team_info = i['team'].serialized()

        # add an dditional key for the year so it is there for reference
        team_info['season'] = file.replace('standings_year_','')

        # add to master record
        all_team_stats.append(team_info)

In [49]:
# create a dataframe and raise all nested json objects to a single row
standings_to_date_df = pd.json_normalize(all_team_stats)

In [62]:
# export result
standings_to_date_df.to_csv(f"{project_dir}\\data\\standings\\standings_to_date.csv", index=False)